In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as D
import torch.optim as Optim

In [ ]:
# import data
with open("./assets/names.txt", mode="r", encoding="utf-8") as file:
    data = file.readlines()
# dataset size
print(len(data))
print(data[:10])
# avg word len
avg_len_words = 0
for word in data:
    avg_len_words += len(word)
print(avg_len_words/len(data))

62262
['Aach im Allgäu\n', 'Aach im Allgäu\n', 'Aach im Allgäu\n', 'Aacherwiese\n', 'Aalbuch\n', 'Aalen\n', 'Aalholz\n', 'Aalkorb\n', 'Abach\n', 'Abbachhof\n']
10.843371558896276


In [28]:
# setup vocabulary
# decision to go with a "1. long streaming approach with multiple names within context" vs "2. one name within context padded to fixed len & special start and end chars"
# i adressed my concerns, that it makes no senses in 1. that via transformer tech some name learns pattern from PREVIOUS names to predict next char
# o1 recommends approach 1 vs. claude recommends approach 2; both unanimous that both ways will net roughly same perplexity, discussion is about top 5%
# maybe i test approach 2 later as comparison
all_chars = list(sorted(set([("".join(char)) for word in data for char in word])))
print(len(all_chars))
print(all_chars)
vocab_size = len(all_chars)

61
['\n', ' ', '-', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'Ä', 'Ö', 'Ü', 'ß', 'ä', 'ö', 'ü']


In [ ]:
# hyperparameters
context_len = 64
n_embd = 256
n_head = 8
n_layer = 6
batch_size = 64
learning_rate = 3e-4